In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
# 입력된 수를 1, 11, 21, 31 ...로 만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기 (검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search, start_pg, end_pg, start_to_end):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&sort=0&photo=0&field=0&pd=3&ds=" + start_date +"&de=" + end_date + "&cluster_rank=10&mynews=1&office_type=1&office_section_code=1&news_office_checked="+ office +"&nso=so:r,p:from"+ start_to_end + ",a:all&start=" + str(start_page)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&sort=0&photo=0&field=0&pd=3&ds=" + start_date +"&de=" + end_date + "&cluster_rank=10&mynews=1&office_type=1&office_section_code=1&news_office_checked="+ office +"&nso=so:r,p:from" + start_to_end + ",a:all&start=" + str(page)
            urls.append(url)
        return urls

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

# html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    # html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url

In [ ]:
# 검색할 엔터테인먼트
search = "YG 엔터테인먼트"

#크롤링 시작 날짜
start_date = "2020.09.01"

#크롤링 마감 날짜
end_date = "2020.12.31"

#크롤링 시작부터 끝
start_to_end = "20201001to20201231"

#언론사 번호
office =  "1005"

In [ ]:
# 검색 시작할 페이지 입력
start_page = 1

# 검색 종료할 페이지 입력
end_page = int(input("크롤링 종료 페이지를 입력(숫자):"))

# naver url 생성
url = makeUrl(search, start_page, end_page, start_to_end)


# 뉴스 크롤러 실행
news_titles = []
news_dates = []

for i in url:
    url_list = articles_crawler(i)  # articles_crawler 함수 수정된 부분
    news_url.append(url_list)

# NAVER 뉴스만 남기기
final_urls = []
for urls in tqdm(news_url):
    for i in urls:
        if "news.naver.com" in i:
            final_urls.append(i)
        else:
            pass


# 뉴스 내용 크롤링
for i in tqdm(final_urls):
    # 각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text, "html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
          title = news_html.select_one("#content > div.end_ct > div > h2")



    # 기사 텍스트만 가져오기
    # list 합치기
    content = ''.join(str(content))

    # html 태그 제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))

    # 날짜 가져오기
    news_dates.append(news_date)

print("\n검색된 기사 갯수: 총 ",(end_page+1-start_page)*10,'개')
print("--------------------크롤링 완료--------------------")

In [ ]:
import pandas as pd

# 데이터 프레임 만들기
news_df = pd.DataFrame({'TITLE':news_titles, 'DATE':news_dates, 'PRESS':news_press, 'REPORTER':news_reporter, 'CONTENT':news_contents})

print("\n------------데이터 전처리 완료-------------")
print('\n           (행, 열) /', news_df.shape)
news_df

# 데이터 프레임 저장
now = datetime.datetime.now()
news_df.to_csv('{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')), encoding='utf-8-sig', index=False)
print("\n----------------저장 완료-----------------")

In [ ]:
news_df.tail(10)